In [1]:
import os
import note_seq
import json
from pathlib import Path

def extract_bach_melodies(bach_midi_folder):

    bach_intervals = []
    processed_files = 0
    
    midi_files = [f for f in os.listdir(bach_midi_folder) if f.endswith('.mid')]
    
    for filename in midi_files:
        filepath = os.path.join(bach_midi_folder, filename)
        
        try:
            sequence = note_seq.midi_file_to_note_sequence(filepath)
            
            if len(sequence.notes) == 0:
                print(f"Warning: {filename}: No notes found")
                continue
            
            # Extract melody line
            melody_intervals = extract_melody_line(sequence)
            
            if melody_intervals:
                bach_intervals.extend(melody_intervals)
                processed_files += 1
                print(f"Success: {filename}: {len(melody_intervals)} intervals extracted")
            else:
                print(f"Warning: {filename}: No melody extracted")
                
        except Exception as e:
            print(f"Error: {filename}: Error - {e}")
    
    print(f"\nBACH CORPUS SUMMARY:")
    print(f"   Processed files: {processed_files}")
    print(f"   Total intervals: {len(bach_intervals)}")
    
    # Save Bach corpus to CURRENT DIRECTORY
    output_path = 'bach_reference_intervals.json'
    with open(output_path, 'w') as f:
        json.dump(bach_intervals, f)
    
    print(f"Saved Bach interval corpus to: {output_path}")
    return bach_intervals

def extract_llm_corpus_intervals(llm_midi_folder):
    """
    Extract interval sequences from your LLM-generated MIDI files.
    Saves JSON to current directory.
    """
    
    print("EXTRACTING LLM CORPUS INTERVALS")
    print("=" * 50)
    
    llm_intervals = []
    processed_files = 0
    
    midi_files = [f for f in os.listdir(llm_midi_folder) if f.endswith('.mid')]
    
    for filename in midi_files:
        filepath = os.path.join(llm_midi_folder, filename)
        
        try:
            # Load MIDI
            sequence = note_seq.midi_file_to_note_sequence(filepath)
            
            if len(sequence.notes) == 0:
                print(f"Warning: {filename}: No notes found")
                continue
            
            # Extract intervals (same as Bach method)
            notes = sorted(sequence.notes, key=lambda n: n.start_time)
            pitches = [note.pitch for note in notes]
            
            if len(pitches) < 2:
                continue
                
            # Convert to intervals
            intervals = []
            for i in range(1, len(pitches)):
                interval = pitches[i] - pitches[i-1]
                intervals.append(interval)
            
            llm_intervals.extend(intervals)
            processed_files += 1
            print(f"Success: {filename}: {len(intervals)} intervals extracted")
            
        except Exception as e:
            print(f"Error: {filename}: Error - {e}")
    
    print(f"\nLLM CORPUS SUMMARY:")
    print(f"   Processed files: {processed_files}")
    print(f"   Total intervals: {len(llm_intervals)}")
    
    # Save LLM corpus to CURRENT DIRECTORY
    output_path = 'llm_bach_intervals.json'
    with open(output_path, 'w') as f:
        json.dump(llm_intervals, f)
    
    print(f"Saved LLM interval corpus to: {output_path}")
    return llm_intervals

def extract_melody_line(sequence):
    """Extract the primary melody line from a MIDI sequence."""
    
    if len(sequence.notes) == 0:
        return []
    
    # Sort notes by start time
    notes = sorted(sequence.notes, key=lambda n: n.start_time)
    
    # Method 1: For solo works, use all notes
    if len(set(note.instrument for note in notes)) == 1:
        # Single instrument - use all notes
        pitches = [note.pitch for note in notes]
    else:
        # Method 2: For polyphonic works, extract highest voice
        # Group notes by time windows and take highest pitch
        pitches = extract_soprano_voice(notes)
    
    # Convert to intervals
    if len(pitches) < 2:
        return []
    
    intervals = []
    for i in range(1, len(pitches)):
        interval = pitches[i] - pitches[i-1]
        intervals.append(interval)
    
    return intervals

def extract_soprano_voice(notes):
    """Extract the soprano (highest) voice from polyphonic MIDI."""
    
    # Group notes by time windows (handle simultaneity)
    time_windows = {}
    window_size = 0.1  # 100ms window
    
    for note in notes:
        window = round(note.start_time / window_size) * window_size
        if window not in time_windows:
            time_windows[window] = []
        time_windows[window].append(note)
    
    # Extract highest pitch from each time window
    soprano_pitches = []
    for time_point in sorted(time_windows.keys()):
        notes_at_time = time_windows[time_point]
        highest_note = max(notes_at_time, key=lambda n: n.pitch)
        soprano_pitches.append(highest_note.pitch)
    
    return soprano_pitches


bach_intervals = extract_bach_melodies('bach_reference/')
llm_intervals = extract_llm_corpus_intervals('Generated_corpus/')


/home/koehler.ale/miniconda3/envs/llm-music/lib/python3.10/site-packages/pydub/utils.py:170: RuntimeWarning: Couldn't find ffmpeg or avconv - defaulting to ffmpeg, but may not work
  warn("Couldn't find ffmpeg or avconv - defaulting to ffmpeg, but may not work", RuntimeWarning)


Success: invent1.mid: 341 intervals extracted
Success: invent5.mid: 529 intervals extracted
Success: invent6.mid: 885 intervals extracted
Success: invent7.mid: 384 intervals extracted
Success: invent8.mid: 372 intervals extracted
Success: invent9.mid: 401 intervals extracted

BACH CORPUS SUMMARY:
   Processed files: 6
   Total intervals: 2912
Saved Bach interval corpus to: bach_reference_intervals.json
EXTRACTING LLM CORPUS INTERVALS
Success: compose_a_melody_in_000.mid: 31 intervals extracted
Success: compose_a_melody_in_001.mid: 37 intervals extracted
Success: compose_a_melody_in_002.mid: 35 intervals extracted
Success: compose_a_melody_in_003.mid: 34 intervals extracted
Success: compose_a_melody_in_004.mid: 35 intervals extracted
Success: compose_a_melody_in_005.mid: 35 intervals extracted
Success: compose_a_melody_in_006.mid: 37 intervals extracted
Success: compose_a_melody_in_007.mid: 33 intervals extracted
Success: compose_a_melody_in_008.mid: 37 intervals extracted
Success: comp

In [3]:
from collections import Counter, defaultdict

def generate_ngrams(interval_sequence, n=3):
    """Generate n-grams from interval sequence."""
    
    if len(interval_sequence) < n:
        return []
    
    ngrams = []
    for i in range(len(interval_sequence) - n + 1):
        ngram = tuple(interval_sequence[i:i+n])
        ngrams.append(ngram)
    
    return ngrams

def analyze_bach_vs_llm_ngrams(bach_json_file, llm_json_file, n=3):
    """
    Compare Bach vs LLM melodic patterns using n-gram analysis.
    
    This is the core research function!
    """
    
    print(f"N-GRAM STYLE AUTHENTICITY ANALYSIS (n={n})")
    print("=" * 60)
    
    # Load both corpora
    print("Loading corpora...")
    with open(bach_json_file, 'r') as f:
        bach_intervals = json.load(f)
    
    with open(llm_json_file, 'r') as f:
        llm_intervals = json.load(f)
    
    print(f"   Bach corpus: {len(bach_intervals)} intervals")
    print(f"   LLM corpus: {len(llm_intervals)} intervals")
    
    # Generate n-grams
    print(f"\nGenerating {n}-grams...")
    bach_ngrams = generate_ngrams(bach_intervals, n)
    llm_ngrams = generate_ngrams(llm_intervals, n)
    
    print(f"   Bach {n}-grams: {len(bach_ngrams)}")
    print(f"   LLM {n}-grams: {len(llm_ngrams)}")
    
    # Convert to frequency distributions
    bach_freq = Counter(bach_ngrams)
    llm_freq = Counter(llm_ngrams)
    
    # Calculate overlap
    bach_patterns = set(bach_freq.keys())
    llm_patterns = set(llm_freq.keys())
    shared_patterns = bach_patterns.intersection(llm_patterns)
    
    # Authenticity metrics
    pattern_overlap = len(shared_patterns) / len(llm_patterns) * 100 if llm_patterns else 0
    bach_coverage = len(shared_patterns) / len(bach_patterns) * 100 if bach_patterns else 0
    
    print(f"\nAUTHENTICITY ANALYSIS")
    print("=" * 30)
    print(f"LLM Pattern Authenticity: {pattern_overlap:.1f}%")
    print(f"   ({len(shared_patterns)} of {len(llm_patterns)} LLM patterns found in Bach)")
    print(f"Bach Pattern Coverage: {bach_coverage:.1f}%")
    print(f"   ({len(shared_patterns)} of {len(bach_patterns)} Bach patterns reproduced)")
    
    # Most authentic LLM patterns
    print(f"\nMOST BACH-LIKE LLM PATTERNS:")
    authentic_patterns = []
    for pattern in shared_patterns:
        bach_count = bach_freq[pattern]
        llm_count = llm_freq[pattern]
        authenticity_score = min(bach_count, llm_count)  # How well matched
        authentic_patterns.append((pattern, bach_count, llm_count, authenticity_score))
    
    # Sort by authenticity score
    authentic_patterns.sort(key=lambda x: x[3], reverse=True)
    
    for i, (pattern, bach_count, llm_count, score) in enumerate(authentic_patterns[:10]):
        interval_str = " ".join([f"{'+' if x >= 0 else ''}{x}" for x in pattern])
        print(f"   {i+1:2d}. {interval_str:15} | Bach: {bach_count:3} | LLM: {llm_count:3}")
    
    # Most un-Bach-like LLM patterns
    print(f"\nLEAST BACH-LIKE LLM PATTERNS:")
    unauthentic_patterns = [(pattern, llm_freq[pattern]) for pattern in (llm_patterns - shared_patterns)]
    unauthentic_patterns.sort(key=lambda x: x[1], reverse=True)
    
    for i, (pattern, count) in enumerate(unauthentic_patterns[:5]):
        interval_str = " ".join([f"{'+' if x >= 0 else ''}{x}" for x in pattern])
        print(f"   {i+1}. {interval_str:15} | Used {count} times (not in Bach)")
    
    # Overall assessment
    print(f"\nOVERALL BACH AUTHENTICITY SCORE")
    print("=" * 35)
    
    overall_score = (pattern_overlap + bach_coverage) / 2
    print(f"Composite Score: {overall_score:.1f}/100")
    
    return {
        'pattern_overlap': pattern_overlap,
        'bach_coverage': bach_coverage,
        'overall_score': overall_score,
        'shared_patterns': len(shared_patterns),
        'bach_unique_patterns': len(bach_patterns - shared_patterns),
        'llm_unique_patterns': len(llm_patterns - shared_patterns),
        'most_authentic': authentic_patterns[:10],
        'least_authentic': unauthentic_patterns[:5]
    }

def quick_comparison_test(bach_json, llm_json):
    """Quick test of different n-gram sizes."""
    
    print("TESTING DIFFERENT N-GRAM SIZES")
    print("=" * 40)
    
    for n in [2, 3, 4, 5]:
        print(f"\nTesting {n}-grams:")
        result = analyze_bach_vs_llm_ngrams(bach_json, llm_json, n=n)
        print(f"   Authenticity: {result['overall_score']:.1f}%")

analyze_bach_vs_llm_ngrams('bach_reference_intervals.json', 'llm_bach_intervals.json')


N-GRAM STYLE AUTHENTICITY ANALYSIS (n=3)
Loading corpora...
   Bach corpus: 2912 intervals
   LLM corpus: 2065 intervals

Generating 3-grams...
   Bach 3-grams: 2910
   LLM 3-grams: 2063

AUTHENTICITY ANALYSIS
LLM Pattern Authenticity: 13.3%
   (81 of 609 LLM patterns found in Bach)
Bach Pattern Coverage: 6.0%
   (81 of 1346 Bach patterns reproduced)

MOST BACH-LIKE LLM PATTERNS:
    1. -2 -2 -1        | Bach:  47 | LLM:  56
    2. -2 -1 -2        | Bach:  39 | LLM:  37
    3. -2 -2 +2        | Bach:  15 | LLM:  10
    4. -1 -2 -2        | Bach:  45 | LLM:  10
    5. +2 -2 -2        | Bach:  12 | LLM:   8
    6. -2 -2 -2        | Bach:   8 | LLM:  39
    7. -2 -1 +3        | Bach:   7 | LLM:  30
    8. -2 +4 -2        | Bach:   9 | LLM:   6
    9. -2 +2 -3        | Bach:  18 | LLM:   5
   10. -1 -2 +3        | Bach:   8 | LLM:   5

LEAST BACH-LIKE LLM PATTERNS:
   1. +4 +3 +4        | Used 94 times (not in Bach)
   2. +4 +1 +4        | Used 34 times (not in Bach)
   3. +1 +4 +3        

{'pattern_overlap': 13.30049261083744,
 'bach_coverage': 6.017830609212481,
 'overall_score': 9.65916161002496,
 'shared_patterns': 81,
 'bach_unique_patterns': 1265,
 'llm_unique_patterns': 528,
 'most_authentic': [((-2, -2, -1), 47, 56, 47),
  ((-2, -1, -2), 39, 37, 37),
  ((-2, -2, 2), 15, 10, 10),
  ((-1, -2, -2), 45, 10, 10),
  ((2, -2, -2), 12, 8, 8),
  ((-2, -2, -2), 8, 39, 8),
  ((-2, -1, 3), 7, 30, 7),
  ((-2, 4, -2), 9, 6, 6),
  ((-2, 2, -3), 18, 5, 5),
  ((-1, -2, 3), 8, 5, 5)],
 'least_authentic': [((4, 3, 4), 94),
  ((4, 1, 4), 34),
  ((1, 4, 3), 33),
  ((3, 4, -4), 26),
  ((4, -11, 4), 22)]}

In [4]:
import json
import note_seq
import numpy as np
from collections import Counter, defaultdict
import os

def calculate_ratio_similarity(bach_value, llm_value):
    """
    Calculate proper ratio-based similarity instead of difference-based.
    Returns the proportion that LLM achieved relative to Bach (0-100%).
    """
    if bach_value == 0 and llm_value == 0:
        return 100.0  # Both zero = perfect match
    elif bach_value == 0:
        return 0.0    # Bach has none, LLM has some = no similarity
    else:
        # Calculate how much LLM achieved relative to Bach
        ratio = llm_value / bach_value
        # Cap at 100% (can't be more similar than perfect match)
        # But if LLM exceeds Bach significantly, similarity decreases
        if ratio <= 1.0:
            return ratio * 100
        else:
            # Penalize overshooting Bach's pattern
            return max(0, 100 - ((ratio - 1) * 100))

def calculate_distribution_similarity(bach_dist, llm_dist, top_n=5):
    """
    Calculate similarity based on how well LLM reproduces Bach's most important intervals.
    """
    total_similarity = 0
    bach_total = sum(bach_dist.values())
    llm_total = sum(llm_dist.values())
    
    # Get Bach's top N most important patterns
    bach_top = bach_dist.most_common(top_n)
    
    for interval, bach_count in bach_top:
        bach_percentage = (bach_count / bach_total) * 100
        llm_count = llm_dist.get(interval, 0)
        llm_percentage = (llm_count / llm_total) * 100 if llm_total > 0 else 0
        
        reproduction_score = calculate_ratio_similarity(bach_percentage, llm_percentage)
        total_similarity += reproduction_score
    
    return total_similarity / top_n

def comprehensive_musical_analysis(bach_json='bach_reference_intervals.json', 
                                           llm_json='llm_bach_intervals.json', 
                                           bach_midi_folder=None, 
                                           llm_midi_folder=None):
    """
    Comprehensive analysis using proper ratio-based similarity calculations.
    """
    
    print("COMPREHENSIVE MUSICAL ANALYSIS")
    print("=" * 60)
    print("Using ratio-based similarity calculations")
    print(f"Bach Reference: {bach_json}")
    print(f"LLM Generated: {llm_json}")
    
    # Load interval data
    try:
        with open(bach_json, 'r') as f:
            bach_data = json.load(f)
        print(f"Loaded Bach reference data")
    except Exception as e:
        print(f"Error loading Bach data: {e}")
        return None
        
    try:
        with open(llm_json, 'r') as f:
            llm_data = json.load(f)
        print(f"Loaded LLM generated data")
    except Exception as e:
        print(f"Error loading LLM data: {e}")
        return None
    
    # Extract intervals
    if isinstance(bach_data, list):
        bach_intervals = bach_data
    elif isinstance(bach_data, dict):
        if 'intervals' in bach_data:
            bach_intervals = bach_data['intervals']
        elif 'interval_sequences' in bach_data:
            bach_intervals = bach_data['interval_sequences']
        else:
            bach_intervals = []
            for value in bach_data.values():
                if isinstance(value, list):
                    bach_intervals.extend(value)
    
    if isinstance(llm_data, list):
        llm_intervals = llm_data
    elif isinstance(llm_data, dict):
        if 'intervals' in llm_data:
            llm_intervals = llm_data['intervals']
        elif 'interval_sequences' in llm_data:
            llm_intervals = llm_data['interval_sequences']
        else:
            llm_intervals = []
            for value in llm_data.values():
                if isinstance(value, list):
                    llm_intervals.extend(value)
    
    print(f"Data loaded: {len(bach_intervals)} Bach intervals, {len(llm_intervals)} LLM intervals")
    
    results = {}
    
    # Analysis 1: Stepwise Motion
    print("\n1. STEPWISE MOTION ANALYSIS")
    print("-" * 45)
    
    bach_interval_dist = Counter(bach_intervals)
    llm_interval_dist = Counter(llm_intervals)
    
    stepwise_intervals = [-2, -1, +1, +2]
    bach_stepwise = sum(bach_interval_dist[i] for i in stepwise_intervals)
    llm_stepwise = sum(llm_interval_dist[i] for i in stepwise_intervals)
    
    bach_stepwise_pct = (bach_stepwise / len(bach_intervals)) * 100
    llm_stepwise_pct = (llm_stepwise / len(llm_intervals)) * 100
    
    stepwise_similarity = calculate_ratio_similarity(bach_stepwise_pct, llm_stepwise_pct)
    
    print(f"Stepwise Motion (±1,±2 semitones):")
    print(f"   Bach: {bach_stepwise_pct:.1f}% ({bach_stepwise}/{len(bach_intervals)})")
    print(f"   LLM:  {llm_stepwise_pct:.1f}% ({llm_stepwise}/{len(llm_intervals)})")
    print(f"   Ratio: {llm_stepwise_pct/bach_stepwise_pct:.2f}x Bach's rate")
    print(f"   Similarity: {stepwise_similarity:.1f}%")
    
    results['stepwise_analysis'] = {
        'bach_stepwise_pct': bach_stepwise_pct,
        'llm_stepwise_pct': llm_stepwise_pct,
        'stepwise_similarity': stepwise_similarity,
        'ratio_multiplier': llm_stepwise_pct / bach_stepwise_pct
    }
    
    # Analysis 2: Leap Patterns
    print(f"\n2. LEAP PATTERN ANALYSIS")
    print("-" * 35)
    
    large_leaps = [i for i in range(-12, 13) if abs(i) > 3]
    bach_leaps = sum(bach_interval_dist[i] for i in large_leaps)
    llm_leaps = sum(llm_interval_dist[i] for i in large_leaps)
    
    bach_leap_pct = (bach_leaps / len(bach_intervals)) * 100
    llm_leap_pct = (llm_leaps / len(llm_intervals)) * 100
    
    leap_similarity = calculate_ratio_similarity(bach_leap_pct, llm_leap_pct)
    if llm_leap_pct > bach_leap_pct:
        excess_penalty = (llm_leap_pct - bach_leap_pct) / bach_leap_pct
        leap_similarity = max(0, leap_similarity - (excess_penalty * 50))
    
    print(f"Large Leaps (>3 semitones):")
    print(f"   Bach: {bach_leap_pct:.1f}% ({bach_leaps}/{len(bach_intervals)})")
    print(f"   LLM:  {llm_leap_pct:.1f}% ({llm_leaps}/{len(llm_intervals)})")
    print(f"   Ratio: {llm_leap_pct/bach_leap_pct:.2f}x Bach's rate")
    print(f"   Similarity: {leap_similarity:.1f}%")
    
    results['leap_analysis'] = {
        'bach_leap_pct': bach_leap_pct,
        'llm_leap_pct': llm_leap_pct,
        'leap_similarity': leap_similarity,
        'ratio_multiplier': llm_leap_pct / bach_leap_pct
    }
    
    # Analysis 3: Direction Changes
    print(f"\n3. MELODIC CONTOUR ANALYSIS")
    print("-" * 40)
    
    bach_direction_changes = count_direction_changes(bach_intervals)
    llm_direction_changes = count_direction_changes(llm_intervals)
    
    bach_direction_pct = (bach_direction_changes / (len(bach_intervals)-1)) * 100 if len(bach_intervals) > 1 else 0
    llm_direction_pct = (llm_direction_changes / (len(llm_intervals)-1)) * 100 if len(llm_intervals) > 1 else 0
    
    # CORRECTED CALCULATION
    contour_similarity = calculate_ratio_similarity(bach_direction_pct, llm_direction_pct)
    
    print(f"Direction Changes (melodic peaks/valleys):")
    print(f"   Bach: {bach_direction_pct:.1f}% ({bach_direction_changes} changes)")
    print(f"   LLM:  {llm_direction_pct:.1f}% ({llm_direction_changes} changes)")
    print(f"   Ratio: {llm_direction_pct/bach_direction_pct:.2f}x Bach's rate")
    print(f"   Similarity: {contour_similarity:.1f}%")
    
    results['contour_analysis'] = {
        'bach_direction_pct': bach_direction_pct,
        'llm_direction_pct': llm_direction_pct,
        'contour_similarity': contour_similarity,
        'ratio_multiplier': llm_direction_pct / bach_direction_pct
    }
    
    # Analysis 4: Top Interval Distribution Matching
    print(f"\n4. BACH'S TOP INTERVAL REPRODUCTION")
    print("-" * 40)
    
    # How well does LLM reproduce Bach's top patterns?
    bach_top_5 = bach_interval_dist.most_common(5)
    print(f"Bach's Top 5 Intervals vs LLM Reproduction:")
    
    interval_reproduction_scores = []
    for i, (interval, bach_count) in enumerate(bach_top_5, 1):
        bach_percentage = (bach_count / len(bach_intervals)) * 100
        llm_count = llm_interval_dist.get(interval, 0)
        llm_percentage = (llm_count / len(llm_intervals)) * 100 if len(llm_intervals) > 0 else 0
        
        reproduction_score = calculate_ratio_similarity(bach_percentage, llm_percentage)
        interval_reproduction_scores.append(reproduction_score)
        
        print(f"   {i}. {interval:+3d}: Bach {bach_percentage:4.1f}% -> LLM {llm_percentage:4.1f}% = {reproduction_score:4.1f}% similarity")
    
    avg_interval_reproduction = sum(interval_reproduction_scores) / len(interval_reproduction_scores)
    
    results['interval_reproduction'] = {
        'bach_top_intervals': bach_top_5,
        'individual_scores': interval_reproduction_scores,
        'avg_reproduction_score': avg_interval_reproduction
    }
    
    # Analysis 5: Chromatic Content
    print(f"\n5. CHROMATIC CONTENT ANALYSIS")
    print("-" * 40)
    
    chromatic = [-1, +1]
    bach_chromatic = sum(1 for i in bach_intervals if i in chromatic)
    llm_chromatic = sum(1 for i in llm_intervals if i in chromatic)
    
    bach_chromatic_pct = (bach_chromatic / len(bach_intervals)) * 100
    llm_chromatic_pct = (llm_chromatic / len(llm_intervals)) * 100
    
    # CORRECTED CALCULATION
    chromatic_similarity = calculate_ratio_similarity(bach_chromatic_pct, llm_chromatic_pct)
    
    print(f"Chromatic Content (±1 semitone):")
    print(f"   Bach: {bach_chromatic_pct:.1f}%")
    print(f"   LLM:  {llm_chromatic_pct:.1f}%")
    print(f"   Ratio: {llm_chromatic_pct/bach_chromatic_pct:.2f}x Bach's rate")
    print(f"   Similarity: {chromatic_similarity:.1f}%")
    
    results['chromatic_analysis'] = {
        'bach_chromatic_pct': bach_chromatic_pct,
        'llm_chromatic_pct': llm_chromatic_pct,
        'chromatic_similarity': chromatic_similarity,
        'ratio_multiplier': llm_chromatic_pct / bach_chromatic_pct
    }
    
    # Analysis 6: LLM's Non-Bach Patterns
    print(f"\n6. LLM'S NON-BACH PATTERN ANALYSIS")
    print("-" * 40)
    
    # Find LLM's most used patterns that Bach rarely/never used
    llm_top_patterns = llm_interval_dist.most_common(10)
    non_bach_patterns = []
    
    print("LLM's top patterns Bach doesn't use:")
    for interval, llm_count in llm_top_patterns:
        bach_count = bach_interval_dist.get(interval, 0)
        llm_pct = (llm_count / len(llm_intervals)) * 100
        bach_pct = (bach_count / len(bach_intervals)) * 100
        
        # If LLM uses this pattern >3x more than Bach, it's "non-Bach"
        if bach_count == 0 or (llm_pct / bach_pct) > 3:
            non_bach_patterns.append((interval, llm_pct, bach_pct))
    
    for interval, llm_pct, bach_pct in non_bach_patterns[:5]:
        print(f"   {interval:+3d}: LLM {llm_pct:4.1f}% vs Bach {bach_pct:4.1f}%")
    
    # Calculate contamination score
    total_non_bach = sum(llm_pct for _, llm_pct, _ in non_bach_patterns)
    contamination_score = max(0, 100 - total_non_bach)
    
    print(f"\n   Non-Bach contamination: {total_non_bach:.1f}%")
    print(f"   Bach purity score: {contamination_score:.1f}%")
    
    results['non_bach_analysis'] = {
        'non_bach_patterns': non_bach_patterns,
        'contamination_score': total_non_bach,
        'purity_score': contamination_score
    }
    
    # Overall Assessment
    print(f"\nAUTHENTICITY ASSESSMENT")
    print("=" * 50)
    
    # Use only the most important metrics with proper weighting
    core_metrics = [
        ('Stepwise Motion Reproduction', results['stepwise_analysis']['stepwise_similarity'], 0.3),
        ('Top Interval Reproduction', results['interval_reproduction']['avg_reproduction_score'], 0.3),
        ('Leap Restraint', results['leap_analysis']['leap_similarity'], 0.2),
        ('Contour Matching', results['contour_analysis']['contour_similarity'], 0.1),
        ('Chromatic Reproduction', results['chromatic_analysis']['chromatic_similarity'], 0.1)
    ]
    
    weighted_score = sum(score * weight for _, score, weight in core_metrics)
    results['corrected_overall_score'] = weighted_score
    
    print("Individual Metric Scores:")
    for metric_name, score, weight in core_metrics:
        print(f"   {metric_name:<25}: {score:5.1f}% (weight: {weight:.1f})")
    
    print(f"\nOverall Authenticity Score: {weighted_score:.1f}%")
    
    if weighted_score > 75:
        print("EXCELLENT: Strong Bach-style authenticity!")
    elif weighted_score > 60:
        print("GOOD: Moderate Bach-style characteristics")
    elif weighted_score > 45:
        print("MIXED: Some Bach elements but significant gaps")
    elif weighted_score > 30:
        print("WEAK: Limited Bach-style authenticity")
    else:
        print("POOR: Minimal Bach-style learning")
    
    # Comparison with old method
    old_method_scores = [84.9, 76.0, 79.6, 89.2]  # From your original results
    old_average = sum(old_method_scores) / len(old_method_scores)
    
    print(f"\nCOMPARISON:")
    print(f"   Old (flawed) method: {old_average:.1f}%")
    print(f"   New method: {weighted_score:.1f}%")
    print(f"   Difference: {old_average - weighted_score:+.1f}%")
    
    return results

def count_direction_changes(intervals):
    """Count how often melody changes direction."""
    if len(intervals) < 2:
        return 0
    
    direction_changes = 0
    for i in range(1, len(intervals)):
        prev_direction = 1 if intervals[i-1] > 0 else -1 if intervals[i-1] < 0 else 0
        curr_direction = 1 if intervals[i] > 0 else -1 if intervals[i] < 0 else 0
        
        if prev_direction != 0 and curr_direction != 0 and prev_direction != curr_direction:
            direction_changes += 1
    
    return direction_changes

def run_comprehensive_analysis(bach_json='bach_reference_intervals.json', 
                                       llm_json='llm_bach_intervals.json'):
    """
    Run the comprehensive analysis.
    """
    
    print("RUNNING COMPREHENSIVE MUSICAL ANALYSIS")
    print("=" * 60)
    print("Using proper ratio-based similarity calculations")
    print("Weighted scoring based on musical importance")
    print()
    
    results = comprehensive_musical_analysis(bach_json, llm_json)
    
    if results is None:
        print("Analysis failed")
        return None
    
    print(f"\nFINAL ASSESSMENT")
    print("=" * 40)
    print(f" {results['corrected_overall_score']:.1f}% Bach authenticity")
    
    return results

results = run_comprehensive_analysis()


RUNNING COMPREHENSIVE MUSICAL ANALYSIS
Using proper ratio-based similarity calculations
Weighted scoring based on musical importance

COMPREHENSIVE MUSICAL ANALYSIS
Using ratio-based similarity calculations
Bach Reference: bach_reference_intervals.json
LLM Generated: llm_bach_intervals.json
Loaded Bach reference data
Loaded LLM generated data
Data loaded: 2912 Bach intervals, 2065 LLM intervals

1. STEPWISE MOTION ANALYSIS
---------------------------------------------
Stepwise Motion (±1,±2 semitones):
   Bach: 43.3% (1260/2912)
   LLM:  28.2% (582/2065)
   Ratio: 0.65x Bach's rate
   Similarity: 65.1%

2. LEAP PATTERN ANALYSIS
-----------------------------------
Large Leaps (>3 semitones):
   Bach: 20.1% (586/2912)
   LLM:  44.1% (911/2065)
   Ratio: 2.19x Bach's rate
   Similarity: 0.0%

3. MELODIC CONTOUR ANALYSIS
----------------------------------------
Direction Changes (melodic peaks/valleys):
   Bach: 68.3% (1987 changes)
   LLM:  47.8% (987 changes)
   Ratio: 0.70x Bach's rate


In [ ]:
import json
import matplotlib.pyplot as plt
import numpy as np
from collections import Counter

def visualize_interval_distributions(bach_json, llm_json):
    """Compare interval usage patterns between Bach and LLM."""
    
    # Load data
    with open(bach_json, 'r') as f:
        bach_intervals = json.load(f)
    with open(llm_json, 'r') as f:
        llm_intervals = json.load(f)
    
    # Create frequency distributions
    bach_freq = Counter(bach_intervals)
    llm_freq = Counter(llm_intervals)
    
    # Get all intervals used by either
    all_intervals = sorted(set(list(bach_freq.keys()) + list(llm_freq.keys())))
    
    # Create comparison data
    bach_percentages = [(bach_freq[i] / len(bach_intervals)) * 100 for i in all_intervals]
    llm_percentages = [(llm_freq[i] / len(llm_intervals)) * 100 for i in all_intervals]
    
    # Create visualization
    fig, (ax1, ax2) = plt.subplots(2, 1, figsize=(15, 10))
    
    # Plot 1: Side-by-side comparison - ONLY show significant intervals
    significant_intervals = [i for i in all_intervals if bach_freq[i] > 10 or llm_freq[i] > 3]
    sig_bach_pct = [(bach_freq[i] / len(bach_intervals)) * 100 for i in significant_intervals]
    sig_llm_pct = [(llm_freq[i] / len(llm_intervals)) * 100 for i in significant_intervals]
    
    x_pos = np.arange(len(significant_intervals))
    width = 0.35
    
    ax1.bar(x_pos - width/2, sig_bach_pct, width, label='Bach', alpha=0.8, color='blue')
    ax1.bar(x_pos + width/2, sig_llm_pct, width, label='LLM', alpha=0.8, color='red')
    
    ax1.set_xlabel('Interval (semitones)')
    ax1.set_ylabel('Usage Percentage (%)')
    ax1.set_title('Interval Distribution Comparison: Bach vs LLM (Significant Intervals Only)')
    ax1.set_xticks(x_pos)
    ax1.set_xticklabels([f'{i:+d}' for i in significant_intervals], rotation=0, fontsize=12)
    ax1.legend()
    ax1.grid(True, alpha=0.3)
    
    # Plot 2: Focus on stepwise intervals
    stepwise_intervals = [-5, -4, -3, -2, -1, 0, +1, +2, +3, +4, +5]
    stepwise_bach = [bach_freq[i] / len(bach_intervals) * 100 for i in stepwise_intervals]
    stepwise_llm = [llm_freq[i] / len(llm_intervals) * 100 for i in stepwise_intervals]
    
    x_step = np.arange(len(stepwise_intervals))
    ax2.bar(x_step - width/2, stepwise_bach, width, label='Bach', alpha=0.8, color='blue')
    ax2.bar(x_step + width/2, stepwise_llm, width, label='LLM', alpha=0.8, color='red')
    
    ax2.set_xlabel('Interval (semitones)')
    ax2.set_ylabel('Usage Percentage (%)')
    ax2.set_title('Stepwise Motion Comparison (±5 semitones)')
    ax2.set_xticks(x_step)
    ax2.set_xticklabels([f'{i:+d}' for i in stepwise_intervals])
    ax2.legend()
    ax2.grid(True, alpha=0.3)
    
    plt.tight_layout()
    plt.savefig('interval_distribution_comparison.png', dpi=300, bbox_inches='tight')
    plt.show()
    
    print("Visualization saved as: interval_distribution_comparison.png")

def visualize_melodic_contours(bach_json, llm_json, sample_length=50):
    """Visualize actual melodic contours to see the difference."""
    
    # Load data
    with open(bach_json, 'r') as f:
        bach_intervals = json.load(f)
    with open(llm_json, 'r') as f:
        llm_intervals = json.load(f)
    
    # Convert intervals back to pitch sequences for visualization
    def intervals_to_pitches(intervals, start_pitch=60):
        pitches = [start_pitch]
        for interval in intervals:
            pitches.append(pitches[-1] + interval)
        return pitches
    
    # Get sample melodies
    bach_sample = intervals_to_pitches(bach_intervals[:sample_length])
    llm_sample = intervals_to_pitches(llm_intervals[:sample_length])
    
    # Create visualization
    fig, (ax1, ax2) = plt.subplots(2, 1, figsize=(15, 8))
    
    # Bach melody contour
    ax1.plot(bach_sample, 'b-', linewidth=2, marker='o', markersize=3)
    ax1.set_title(f'Bach Melodic Contour (First {sample_length} intervals)')
    ax1.set_ylabel('MIDI Pitch')
    ax1.grid(True, alpha=0.3)
    ax1.set_ylim(45, 85)  # Reasonable pitch range
    
    # LLM melody contour  
    ax2.plot(llm_sample, 'r-', linewidth=2, marker='o', markersize=3)
    ax2.set_title(f'LLM Melodic Contour (First {sample_length} intervals)')
    ax2.set_xlabel('Note Position')
    ax2.set_ylabel('MIDI Pitch')
    ax2.grid(True, alpha=0.3)
    ax2.set_ylim(45, 85)  # Same range for comparison
    
    plt.tight_layout()
    plt.savefig('melodic_contour_comparison.png', dpi=300, bbox_inches='tight')
    plt.show()
    
    print("Visualization saved as: melodic_contour_comparison.png")



def analyze_problematic_patterns(bach_json, llm_json):
    """Deep dive into why the LLM fails to match Bach."""
    
    print("\nDEEP DIVE: WHY LLM FAILS TO MATCH BACH")
    print("=" * 50)
    
    # Load data
    with open(bach_json, 'r') as f:
        bach_intervals = json.load(f)
    with open(llm_json, 'r') as f:
        llm_intervals = json.load(f)
    
    bach_freq = Counter(bach_intervals)
    llm_freq = Counter(llm_intervals)
    
    print("INTERVALS LLM OVERUSES (vs Bach):")
    llm_only_intervals = []
    for interval, llm_count in llm_freq.most_common(10):
        llm_pct = (llm_count / len(llm_intervals)) * 100
        bach_pct = (bach_freq[interval] / len(bach_intervals)) * 100
        overuse = llm_pct - bach_pct
        
        if overuse > 5:  # More than 5% overuse
            llm_only_intervals.append((interval, llm_pct, bach_pct, overuse))
    
    for interval, llm_pct, bach_pct, overuse in llm_only_intervals:
        print(f"    {interval:+3d}: LLM {llm_pct:5.1f}% vs Bach {bach_pct:4.1f}% (+{overuse:4.1f}% overuse)")
    
    print("\nINTERVALS LLM UNDERUSES (missing Bach patterns):")
    missing_patterns = []
    for interval, bach_count in bach_freq.most_common(10):
        bach_pct = (bach_count / len(bach_intervals)) * 100
        llm_pct = (llm_freq[interval] / len(llm_intervals)) * 100
        underuse = bach_pct - llm_pct
        
        if underuse > 5:  # More than 5% underuse
            missing_patterns.append((interval, bach_pct, llm_pct, underuse))
    
    for interval, bach_pct, llm_pct, underuse in missing_patterns:
        print(f"    {interval:+3d}: Bach {bach_pct:5.1f}% vs LLM {llm_pct:4.1f}% (-{underuse:4.1f}% missing)")
    
    return {
        'overused_intervals': llm_only_intervals,
        'underused_intervals': missing_patterns
    }

def create_all_visualizations(bach_json='bach_reference_intervals.json', llm_json='llm_bach_intervals.json'):
    """Create comprehensive visualization suite."""
    
    print("CREATING COMPREHENSIVE VISUALIZATION SUITE")
    print("=" * 50)
    
    # 1. Interval distribution comparison
    print("1. Creating interval distribution comparison...")
    visualize_interval_distributions(bach_json, llm_json)
    
    # 2. Melodic contour comparison  
    print("2. Creating melodic contour comparison...")
    visualize_melodic_contours(bach_json, llm_json)
    

    
    # 3. Problem analysis
    print("3. Analyzing problematic patterns...")
    problem_analysis = analyze_problematic_patterns(bach_json, llm_json)
    
    print("\nVISUALIZATIONS CREATED:")
    print("   - interval_distribution_comparison.png")
    print("   - melodic_contour_comparison.png")
    
    return problem_analysis

def simple_interval_comparison(bach_json, llm_json):
    """Quick visual comparison of top intervals."""
    
    # Load data
    with open(bach_json, 'r') as f:
        bach_intervals = json.load(f)
    with open(llm_json, 'r') as f:
        llm_intervals = json.load(f)
    
    bach_freq = Counter(bach_intervals)
    llm_freq = Counter(llm_intervals)
    
    # Get top 10 intervals for each
    bach_top = dict(bach_freq.most_common(10))
    llm_top = dict(llm_freq.most_common(10))
    
    print("TOP 10 INTERVALS COMPARISON")
    print("=" * 40)
    print("Interval | Bach %  | LLM %   | Difference")
    print("-" * 40)
    
    all_top_intervals = sorted(set(list(bach_top.keys()) + list(llm_top.keys())))
    
    for interval in all_top_intervals:
        bach_pct = (bach_top.get(interval, 0) / len(bach_intervals)) * 100
        llm_pct = (llm_top.get(interval, 0) / len(llm_intervals)) * 100
        diff = abs(bach_pct - llm_pct)
        
        print(f"   {interval:+3d}   | {bach_pct:5.1f}% | {llm_pct:5.1f}% | {diff:5.1f}%")


create_all_visualizations()

CREATING COMPREHENSIVE VISUALIZATION SUITE
1. Creating interval distribution comparison...
